## 🚀 大语言模型微调实战教程

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/shareAI-lab/lab-handbook/blob/main/LLM_SFT_for_ERNIE4_5_Chinese.ipynb)
[![GitHub](https://img.shields.io/badge/GitHub-View%20on%20GitHub-blue?logo=github)](https://github.com/shareAI-lab/lab-handbook)

### 📘 欢迎来到大语言模型微调之旅！

在这个教程中，我们将一起探索如何将强大的预训练模型转变为您专属的AI助手。无论您是AI初学者还是经验丰富的开发者，这份教程都将以最直观的方式带您掌握模型微调的核心技术。

---

### 🎯 为什么需要模型微调？

想象一下，预训练的大语言模型就像一位博学的通才，虽然知识渊博，但可能不太了解您的特定业务需求。而**模型微调（Fine-tuning）**就是将这位"通才"培养成您领域的"专家"的过程。

#### 核心概念解析：
- **预训练模型**：在海量文本上训练的基础模型，拥有广泛的语言理解能力
- **微调过程**：使用您的领域数据，让模型学习特定任务的模式和知识
- **最终效果**：获得一个既保留通用能力，又精通特定任务的定制化模型

---

### 🛠️ 本教程的技术选型

我们精心选择了一套**高效、易用、资源友好**的技术栈：

#### 1. **Unsloth 加速引擎** ⚡
- 提供 **2倍训练加速**
- 减少 **60%显存占用**
- 让您在普通GPU上也能训练大模型！

#### 2. **LoRA 微调技术** 🎯
- 只训练 **0.1%的参数**，却能达到接近全量微调的效果
- 训练后的适配器文件仅几十MB，易于部署和分享
- 支持多个LoRA适配器灵活切换

#### 3. **4-bit 量化技术** 💾
- 将模型大小压缩至原来的 **1/4**
- 在保持性能的同时，大幅降低硬件要求
- 让 24GB 显存的消费级显卡也能运行 70B 级别的模型

#### 4. **基座模型：百度文心 ERNIE-4.5** 🤖
我们选择了百度最新的 ERNIE-4.5 系列模型作为基座：
- **0.3B 版本**：适合快速实验和学习（本教程默认）
- **21B 版本**：适合生产环境的高性能需求

---

### 📚 学习路线图

```
第一步：环境搭建（10分钟）
   ↓
第二步：模型加载与配置（5分钟）
   ↓
第三步：数据准备与格式化（10分钟）
   ↓
第四步：启动训练（20分钟）
   ↓
第五步：保存与部署（5分钟）
```

让我们开始这段激动人心的AI之旅吧！🎉

## 📦 第一步：环境准备

### 🔧 安装必要的库

首先，我们需要安装几个关键的Python库。这些库将为我们提供模型训练所需的所有功能。

**提示**：安装过程大约需要2-3分钟，可以趁此时间阅读后续的内容说明。

In [ ]:
# ===== 安装核心库 =====
# Unsloth: 我们的训练加速引擎，从源码安装确保最新版本
!pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

# bitsandbytes: 提供模型量化功能，将模型压缩到 4-bit 或 8-bit
# unsloth_zoo: 包含预配置的模型和工具集
!pip install bitsandbytes unsloth_zoo

# transformers: Hugging Face 的核心库，提供模型和训练器
!pip install -U transformers

print("✅ 环境准备完成！所有依赖已成功安装。")

## 🤖 第二步：加载模型并配置 LoRA

### 💡 理解 LoRA：让大模型微调变得简单

在开始代码之前，让我们先理解一下 **LoRA（低秩适配）** 的魔力：

#### 传统微调 vs LoRA 微调

| 对比项 | 传统全量微调 | LoRA 微调 |
|--------|------------|-----------|
| 训练参数量 | 100% (数十亿) | 0.1-1% (数百万) |
| 显存需求 | 极高 (>80GB) | 较低 (8-24GB) |
| 训练时间 | 数天 | 数小时 |
| 模型文件大小 | 完整模型大小 | 仅适配器几十MB |
| 效果 | 最佳 | 接近最佳 |

#### LoRA 的工作原理

想象模型的知识更新就像给房子装修：
- **传统方法**：拆掉重建整个房子（训练所有参数）
- **LoRA方法**：只在关键位置加装"适配器"（训练少量新参数）

具体来说，LoRA 在模型的注意力层旁边添加两个小矩阵（A和B），通过训练这两个小矩阵来调整模型行为：
```
原始输出 = W × 输入
LoRA输出 = W × 输入 + (B × A) × 输入
           ↑原始不变    ↑新增的可训练部分
```

#### 关键参数解释

- **r (rank)**：LoRA矩阵的秩，控制适配器的容量
  - r=8：轻量级微调，适合简单任务
  - r=16：平衡选择（本教程使用）
  - r=32+：复杂任务，需要更多新知识
  
- **lora_alpha**：缩放因子，通常设为 r 的 2倍
- **lora_dropout**：防止过拟合，通常设为 0.05-0.1
- **target_modules**：要添加LoRA的模块
  - "all-linear"：所有线性层（推荐）
  - 也可指定特定层如 ["q_proj", "v_proj"]

In [ ]:
# ===== 步骤 1: 加载预训练模型 =====
from unsloth import FastModel
import torch

# 设置最大序列长度（影响显存占用和训练速度）
MAX_LEN = 4096  # 可根据任务需求调整：2048（对话）、4096（平衡）、8192（长文本）

# 模型选择指南：
# - baidu/ERNIE-4.5-0.3B-PT: 轻量级，适合学习和快速实验（推荐初学者）
# - baidu/ERNIE-4.5-21B-A3B-PT: 生产级，性能强大但需要更多资源

print("🔄 正在加载模型，请稍候...")
model, tokenizer = FastModel.from_pretrained(
    model_name="baidu/ERNIE-4.5-0.3B-PT",  # 可替换为 21B 版本
    # max_seq_length=MAX_LEN,               # 某些版本需要指定
    # load_in_4bit=False,                   # 显存不足时设为 True（QLoRA）
    # load_in_8bit=False,                   # 8-bit 量化选项
    # full_finetuning=False,                # True 为全量微调（需要大显存）
    trust_remote_code=True,                  # ERNIE 模型需要此参数
)
print("✅ 模型加载成功！")

# ===== 步骤 2: 配置 LoRA 适配器 =====
from unsloth import FastLanguageModel

print("🔧 正在配置 LoRA 适配器...")
model = FastLanguageModel.get_peft_model(
    model,
    r=16,                    # LoRA 秩：8（轻量）、16（平衡）、32（复杂任务）
    lora_alpha=32,           # 缩放系数：通常设为 r 的 2 倍
    lora_dropout=0.05,       # Dropout 率：防止过拟合（0.05-0.1）
    target_modules="all-linear",  # 应用 LoRA 的层："all-linear" 或指定层列表
    use_rslora=True,         # 使用改进的 RSLoRA（更稳定）
    # use_gradient_checkpointing=True,  # 节省显存但训练变慢
)

# 打印模型信息
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
all_params = sum(p.numel() for p in model.parameters())
trainable_percent = 100 * trainable_params / all_params

print(f"✅ LoRA 配置完成！")
print(f"📊 模型参数统计：")
print(f"   - 总参数量: {all_params:,}")
print(f"   - 可训练参数: {trainable_params:,}")
print(f"   - 可训练比例: {trainable_percent:.2f}%")

## 📊 第三步：准备训练数据

### 📝 数据格式化的重要性

数据是模型微调的"教材"。就像教学需要合适的教材一样，模型训练也需要正确格式的数据。

#### 为什么数据格式如此重要？

1. **模型理解**：模型在预训练时已经学会了特定的对话格式
2. **性能影响**：错误的格式会导致模型"听不懂"指令
3. **一致性**：保持格式一致能让模型更好地泛化

#### ERNIE 模型的对话格式

ERNIE-4.5 使用特定的对话模板：
```
<|begin_of_sentence|>User: [用户输入]
Assistant: [助手回复]<|end_of_sentence|>
```

这些特殊标记帮助模型区分：
- 对话的开始和结束
- 用户输入和AI回复的界限
- 多轮对话的上下文

#### 数据集选择建议

本教程使用 Microsoft 的 Orca Math 数据集作为示例，但您可以根据需求选择：

- **通用对话**：ShareGPT、Alpaca 等
- **数学推理**：GSM8K、MATH 等
- **代码生成**：CodeAlpaca、StarCoder 等
- **领域特定**：医疗、法律、金融等专业数据集
- **自定义数据**：您自己的业务数据（推荐）

In [ ]:
# ===== 加载和格式化数据集 =====
from datasets import load_dataset

print("📚 正在加载数据集...")
# 使用 Microsoft 的 Orca 数学问题数据集作为示例
# split="train[:1%]" 表示只使用 1% 的数据进行快速演示
# 生产环境建议使用完整数据集或您自己的数据
ds = load_dataset("microsoft/orca-math-word-problems-200k", split="train[:1%]")
print(f"✅ 数据加载完成！共 {len(ds)} 条样本")

# ===== 数据格式化函数 =====
def format_chat(example):
    """
    将原始数据转换为 ERNIE 模型的对话格式
    
    输入格式：
    - question: 用户的问题
    - answer: 期望的回答
    
    输出格式：
    <|begin_of_sentence|>User: [问题]
    Assistant: [回答]<|end_of_sentence|>
    """
    # 构建对话消息列表
    messages = [{"role": "user", "content": example["question"]}]
    
    # 如果有答案，添加助手回复
    if "answer" in example and example["answer"]:
        messages.append({"role": "assistant", "content": example["answer"]})
    
    # 使用分词器的模板功能自动添加特殊标记
    formatted_text = tokenizer.apply_chat_template(
        messages, 
        tokenize=False,  # 不进行分词，只格式化
        add_generation_prompt=False  # 训练时不需要生成提示
    )
    
    return {"text": formatted_text}

# ===== 应用格式化 =====
print("🔄 正在格式化数据...")
ds = ds.map(
    format_chat,  # 应用格式化函数
    remove_columns=ds.column_names,  # 移除原始列，只保留 "text"
    desc="格式化数据"  # 进度条描述
)

# ===== 验证数据格式 =====
print("✅ 数据格式化完成！")
print("\n📝 数据样例（第一条）：")
print("-" * 50)
print(ds[0]["text"][:500])  # 显示前500个字符
print("-" * 50)
print(f"\n💡 提示：确认数据包含正确的特殊标记！")

### 🔍 数据格式验证

从上面的输出可以看到，数据已经被正确格式化为 ERNIE 模型期望的格式：
- `<|begin_of_sentence|>` 标记对话开始
- `User:` 和 `Assistant:` 明确区分角色
- `<|end_of_sentence|>` 标记对话结束

这种格式确保模型能够正确理解任务边界和角色切换。

## 🚂 第四步：配置并启动训练

### ⚙️ 训练参数详解

训练配置是微调成功的关键。让我们详细了解每个参数的作用和调优建议：

#### 🎯 批处理与梯度累积

**有效批大小 = per_device_train_batch_size × gradient_accumulation_steps × GPU数量**

在我们的配置中：
- `per_device_train_batch_size = 1`：每次前向传播处理1个样本
- `gradient_accumulation_steps = 8`：累积8次梯度后更新
- **有效批大小 = 1 × 8 = 8**

💡 **调优建议**：
- 显存充足：增大 `per_device_train_batch_size`
- 显存不足：减小批大小，增加梯度累积步数
- 目标：有效批大小通常在 8-32 之间效果较好

#### 📈 学习率与优化器

- `learning_rate = 1e-4`：LoRA 微调的典型学习率
  - 太高（>5e-4）：训练不稳定，损失震荡
  - 太低（<1e-5）：收敛缓慢，效果不佳
  - 经验值：LoRA 用 1e-4 到 2e-4，全量微调用 1e-5 到 5e-5

- `optim = "adamw_8bit"`：8-bit AdamW 优化器
  - 节省 75% 优化器状态内存
  - 性能损失几乎可以忽略
  - 适合大模型和长序列训练

- `lr_scheduler_type = "linear"`：线性学习率衰减
  - 训练初期学习率高，快速学习
  - 训练后期学习率低，精细调整
  - 其他选择：cosine（余弦衰减）、constant（恒定）

#### 🔄 训练轮数与步数

- `num_train_epochs = 1`：训练轮数
  - 小数据集（<10k）：3-5 轮
  - 中等数据集（10k-100k）：1-3 轮
  - 大数据集（>100k）：1 轮或按步数

- 也可以使用 `max_steps` 替代：
  - 快速测试：50-100 步
  - 正常训练：500-2000 步
  - 深度训练：5000+ 步

#### 💾 检查点保存

- `save_strategy = "steps"`：按步数保存
- `save_steps = 200`：每200步保存一次
  - 便于中断后恢复训练
  - 可以选择最佳检查点
  - 建议：设为总步数的 10-20%

#### 🚀 性能优化

- `fp16 = True`：使用半精度训练
  - 速度提升 2x
  - 显存减少 50%
  - 适合大多数 GPU（V100、T4、RTX 等）

- `bf16 = False`：BFloat16 精度
  - 更好的数值稳定性
  - 需要 A100、H100 等新GPU
  - 如果支持，优先使用 bf16

- `packing = True`：样本打包
  - 将短样本拼接，提高GPU利用率
  - 特别适合长度差异大的数据集
  - 可提升 2-5x 训练速度

#### 📊 监控与调试

- `logging_steps = 5`：每5步记录一次
  - 观察损失下降趋势
  - 及时发现训练问题
  - 生产环境可设为 10-50

- `report_to = "none"`：不上报到外部平台
  - 可选："tensorboard"、"wandb"
  - 便于可视化和团队协作

### 🎓 训练过程解读

启动训练后，您会看到类似这样的输出：
```
Step | Training Loss | Validation Loss
-----|---------------|----------------
10   | 2.345         | -
20   | 1.876         | -
30   | 1.234         | -
```

**正常情况**：
- Loss 持续下降（从 2-3 降到 0.5-1.5）
- 下降速度逐渐放缓
- 没有突然的跳变

**异常情况及解决**：
- Loss 不下降 → 检查数据格式，增大学习率
- Loss 震荡 → 减小学习率，增加梯度累积
- Loss 突增 → 可能过拟合，添加 dropout，减少训练步数

In [ ]:
# ===== 配置训练参数 =====
from trl import SFTTrainer
from transformers import TrainingArguments

print("⚙️ 配置训练参数...")

# 训练参数配置
training_args = TrainingArguments(
    # ===== 批处理配置 =====
    per_device_train_batch_size=1,      # 每个设备的批大小（根据显存调整）
    gradient_accumulation_steps=8,      # 梯度累积步数（有效批大小 = 1 × 8 = 8）
    
    # ===== 学习率配置 =====
    learning_rate=1e-4,                 # 初始学习率（LoRA 通常用 1e-4 到 2e-4）
    lr_scheduler_type="linear",         # 学习率调度器："linear", "cosine", "constant"
    warmup_steps=0,                     # 预热步数（可选，通常为总步数的 10%）
    
    # ===== 训练时长 =====
    num_train_epochs=1,                 # 训练轮数（或使用 max_steps 指定步数）
    # max_steps=100,                    # 最大训练步数（与 num_train_epochs 二选一）
    
    # ===== 优化器配置 =====
    optim="adamw_8bit",                 # 8-bit AdamW 优化器（节省显存）
    weight_decay=0.01,                  # 权重衰减（L2 正则化）
    
    # ===== 精度配置 =====
    fp16=True,                          # 使用 FP16 混合精度（T4、V100 等）
    bf16=False,                         # 使用 BF16（A100、H100 等新 GPU）
    
    # ===== 保存与日志 =====
    output_dir="outputs",               # 输出目录
    save_strategy="steps",              # 保存策略："steps" 或 "epoch"
    save_steps=200,                     # 每 N 步保存一次检查点
    logging_steps=5,                    # 每 N 步记录一次日志
    report_to="none",                   # 报告目标："tensorboard", "wandb" 或 "none"
    
    # ===== 其他设置 =====
    remove_unused_columns=False,        # 保留所有数据列
    seed=42,                            # 随机种子（保证可重复性）
)

# ===== 初始化训练器 =====
print("🚀 初始化 SFT 训练器...")

trainer = SFTTrainer(
    model=model,                        # 配置好 LoRA 的模型
    tokenizer=tokenizer,                # 分词器
    train_dataset=ds,                   # 训练数据集
    dataset_text_field="text",          # 数据集中文本字段的名称
    args=training_args,                 # 训练参数
    max_seq_length=MAX_LEN,             # 最大序列长度
    packing=True,                       # 启用样本打包（提高 GPU 利用率）
    use_cache=False,                    # 禁用缓存（LoRA 训练必需）
)

print("✅ 训练器配置完成！")
print(f"📊 训练信息：")
print(f"   - 有效批大小: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"   - 总训练样本数: {len(ds)}")
print(f"   - 预计训练步数: {len(ds) // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps)}")

# ===== 开始训练 =====
print("\n🏃 开始训练...（这可能需要几分钟）")
print("💡 提示：观察 loss 是否持续下降\n")

# 执行训练
trainer_stats = trainer.train(
    # resume_from_checkpoint=True,      # 从检查点恢复（如果存在）
)

# ===== 训练完成 =====
print("\n🎉 训练完成！")
print(f"📊 训练统计：")
print(f"   - 总训练时间: {trainer_stats.metrics['train_runtime']:.2f} 秒")
print(f"   - 每秒样本数: {trainer_stats.metrics['train_samples_per_second']:.2f}")
print(f"   - 最终 Loss: {trainer_stats.metrics['train_loss']:.4f}")

## 💾 第五步：保存微调后的模型

In [ ]:
# ===== 保存微调后的模型 =====
print("💾 正在保存模型...")

# 保存路径
save_path = "ernie-4.5-0.3b-sft-merged"

# 执行保存（合并 LoRA 权重到基础模型）
model.save_pretrained_merged(
    save_path,                          # 保存路径
    tokenizer,                          # 同时保存分词器
    save_method="merged_16bit",         # 保存方式：
                                       # - "merged_16bit": 合并后保存为 16-bit（推荐）
                                       # - "merged_4bit": 4-bit 量化保存（最小）
                                       # - "lora": 仅保存 LoRA 适配器
)

print(f"✅ 模型保存成功！")
print(f"📁 保存位置: {save_path}/")
print(f"📦 文件说明：")
print(f"   - config.json: 模型配置文件")
print(f"   - model.safetensors: 模型权重文件")
print(f"   - tokenizer.json: 分词器文件")
print(f"   - tokenizer_config.json: 分词器配置")

### 🎉 恭喜！模型训练完成

您的微调模型已保存至：`/content/ernie-4.5-0.3b-sft-merged`

#### 保存选项说明

我们使用了 `save_pretrained_merged` 方法，它提供了几种保存方式：

1. **merged_16bit**（本教程使用）
   - 将 LoRA 权重合并回原始模型
   - 保存为 16-bit 精度
   - 文件大小适中，推理速度快
   - 适合部署使用

2. **merged_4bit**
   - 4-bit 量化保存
   - 文件最小（约原始 1/4）
   - 适合资源受限环境

3. **lora**
   - 仅保存 LoRA 适配器
   - 文件极小（通常 <100MB）
   - 需要原始模型才能使用
   - 适合多任务切换场景

#### 下一步：如何使用您的模型

```python
# 加载微调后的模型进行推理
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(
    "ernie-4.5-0.3b-sft-merged",
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained("ernie-4.5-0.3b-sft-merged")

# 使用模型生成回复
messages = [{"role": "user", "content": "你的问题"}]
text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(text, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=100)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)
```

In [ ]:
# ===== 内存清理（可选） =====
# 如果需要释放 GPU 内存以进行其他任务，运行以下代码

# import gc
# import torch

# # 删除模型和训练器对象
# del model, tokenizer, trainer
# 
# # 强制垃圾回收
# gc.collect()
# 
# # 清空 CUDA 缓存
# if torch.cuda.is_available():
#     torch.cuda.empty_cache()
#     print("✅ GPU 内存已清理！")

## 📚 总结与进阶指南

### 🎯 您已经掌握的技能

通过完成本教程，您已经学会了：
1. ✅ 使用 Unsloth 加速模型训练
2. ✅ 配置和应用 LoRA 微调技术
3. ✅ 准备和格式化训练数据
4. ✅ 调整训练超参数
5. ✅ 保存和部署微调模型

### 🚀 进阶优化建议

#### 1. **数据质量优化**
- 确保数据多样性和平衡性
- 清洗低质量样本
- 增加难例和边界案例
- 使用数据增强技术

#### 2. **超参数调优**
- 使用网格搜索或贝叶斯优化
- 监控验证集性能
- 早停（Early Stopping）防止过拟合
- 学习率预热（Warmup）策略

#### 3. **多任务学习**
- 训练多个 LoRA 适配器
- 实现任务路由机制
- 共享基座模型，节省资源

#### 4. **生产部署**
- 使用 ONNX 或 TensorRT 加速
- 实现批处理推理
- 添加缓存机制
- 监控模型性能指标

### 🔧 常见问题解决

| 问题 | 可能原因 | 解决方案 |
|------|---------|----------|
| OOM（内存溢出） | 批大小过大/序列过长 | 减小批大小/使用梯度累积/启用量化 |
| Loss 不下降 | 学习率不当/数据问题 | 调整学习率/检查数据格式 |
| 训练速度慢 | 未优化配置 | 启用 fp16/使用 packing/升级 Unsloth |
| 效果不理想 | 数据量不足/参数不当 | 增加数据/调整 LoRA rank |

### 📖 推荐学习资源

1. **Unsloth 官方文档**：[github.com/unslothai/unsloth](https://github.com/unslothai/unsloth)
2. **LoRA 原始论文**：[arxiv.org/abs/2106.09685](https://arxiv.org/abs/2106.09685)
3. **Hugging Face 教程**：[huggingface.co/docs/peft](https://huggingface.co/docs/peft)
4. **ERNIE 模型卡片**：[huggingface.co/baidu](https://huggingface.co/baidu)

### 💬 加入社区

- 遇到问题？在 GitHub Issues 提问
- 想分享经验？加入我们的技术社区
- 有改进建议？欢迎提交 Pull Request

---

**祝您在 AI 微调之旅中取得成功！** 🌟

*如果本教程对您有帮助，请给我们的仓库一个 ⭐ Star！*